## 1. 計算所有車站*各村里距離

In [1]:
import pandas as pd
import numpy as np
from geopy import distance

### 讀取所有車站點位


In [2]:
# 讀取車站資料
df_sta = pd.read_csv('../data/all_stations_20210107.csv', encoding='utf-8')
# mrt['Coordinate'] = [i for i in zip(mrt['lat'],mrt['lng'])]
df_sta

,Station_Type,Activated,Station_ID,Station_Name,County,District,T_Status,Num_Mrt,Num_Train,Num_Thsr,Station_Address,lat,lng
0,TpMrt,1,BL01,頂埔站,新北市,土城區,N,1,0,0,新北市土城區中央路4段51之6號,24.960120,121.420500
1,TpMrt,1,BL02,永寧站,新北市,土城區,N,1,0,0,新北市土城區中央路3段105號,24.966820,121.436130
2,TpMrt,1,BL03,土城站,新北市,土城區,N,1,0,0,新北市土城區金城路1段105號,24.973130,121.444320
3,TpMrt,1,BL04,海山站,新北市,土城區,N,1,0,0,新北市土城區海山路39號,24.985305,121.448730
4,TpMrt,1,BL05,亞東醫院站,新北市,板橋區,N,1,0,0,新北市板橋區南雅南路2段17號,24.998280,121.452465
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Train,1,1100,中壢站,桃園市,中壢區,N,0,1,0,桃園市中壢區石頭里中和路139號,24.953730,121.225594
288,Train,1,1110,埔心站,桃園市,楊梅區,N,0,1,0,桃園市楊梅區埔心里永美路208號,24.919588,121.183502
289,Train,1,1120,楊梅站,桃園市,楊梅區,N,0,1,0,桃園市楊梅區楊梅里大成路256號,24.914325,121.146446
290,Train,1,1130,富岡站,桃園市,楊梅區,N,0,1,0,桃園市楊梅區富岡里成功路37號,24.934427,121.083067


In [4]:
print(df_sta.groupby('Activated').size(),'\n')
print(df_sta.columns)

index = df_sta[df_sta.duplicated('Station_Name', keep=False)].index.copy()
print('重複的站點數量:',df_sta.loc[index,'Station_Name'].nunique())
print('重複的站名:',df_sta.loc[index,'Station_Name'].unique())



Activated
0    104
1    188
dtype: int64 

Index(['Station_Type', 'Activated', 'Station_ID', 'Station_Name', 'County',
       'District', 'T_Status', 'Num_Mrt', 'Num_Train', 'Num_Thsr',
       'Station_Address', 'lat', 'lng'],
      dtype='object')
重複的站點數量: 0
重複的站名: []


### 讀取鄰里資料

In [5]:
# 讀取鄰里資料
vill = pd.read_csv('../data/pop_merge.csv', index_col ='Village_ID' ,  encoding='utf-8')
# vill['Coordinate'] = [ i for i in zip(vill['lat'],vill['lng'])]
vill.dropna(inplace=True)
vill.head(3)

,Country,District,Village,Village_Chief,Address,lat,lng,納稅單位,綜合所得總額,所得平均,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
Village_ID,,,,,,,,,,,,,,,,,,,,,
10017010-001,基隆市,中正區,德義里,張建源,基隆市中正區義二路97之 1號,25.132788,121.746435,275.0,23622.5,85.9,...,240,70,41,6,297,347,2.0,13.0,37,35
10017010-002,基隆市,中正區,正義里,駱錫賢,基隆市中正區義二路147號,25.133882,121.747188,393.0,27012.9,68.7,...,396,166,128,13,516,588,3.0,19.0,65,57
10017010-003,基隆市,中正區,信義里,江添丁,基隆市中正區正義路69號,25.134521,121.748392,257.0,19097.9,74.3,...,310,103,100,14,391,405,2.0,11.0,26,41


### 運算流程
1. 計算所有捷運站*里辦公室的距離(n= 2149)  <br>
2. 將每個捷運站存為一個list / numpy array /series <br>
3. 如果距離小於3公里, 則以{里名：距離} 格式 ,存入每個捷運站的list內 <br>
4. 將捷運站名與 各里距離  以{捷運站名：各里距離} 存為json檔案 <br>  


In [6]:
station_vil_dis = {}
for s in range(len(df_sta['Station_ID'])):  # 從1開始 ，台北車站略過未抓
    #     if s>2:break

    vil_dis ={} # 建立空字典 
    for v in range(1,len(vill['Village'])):
        sta_name = df_sta['Station_Name'][s]
        vill_name = (vill['Country'].iloc[v] + 
                     vill['District'].iloc[v] + 
                     vill['Village'].iloc[v])
        
        # 計算 '捷運站' 與 '村里辦公室'的距離
        mrt_cor  = (df_sta['lat'][s],df_sta['lng'][s])
        vill_cor = (vill['lat'].iloc[v],vill['lng'].iloc[v])
        dis = distance.distance( mrt_cor , vill_cor).km 
        
#         print(v+1,f'{sta_name}-{vill_name},distnce:{dis:.3f}',end='\r')
                    
        if dis<=3:  # 當距離小於n的時候才寫入
            vil_dis.update({vill_name : round(dis,3)})
#             print(f'{sta_name}-{vill_name},{dis:.3f}',end='\r')
     
    station_vil_dis.update({sta_name : vil_dis })
    
    # check progess
    progress  = (s+1)/len(df_sta)*100
  
    print(f'progress ：{s+1} , {progress:.2f}％',end='\r') # print out 完成 %數
#     print(f'{sta_name}-{len(vil_dis)}')
    
output = pd.Series(station_vil_dis)
output.to_json('../data/mrt_village_dis.json')

## 二 計算各車站人口結構權重

### 計算權重流程
1. 讀入車站資料，取得所有車站名單 <br>
2. 讀入計算好距離的各車站與各里距離的json檔 <br>
3. 以迴圈方式一一指定各站點名稱 <br>
4. 以站點名稱作為 key,取得各車站與里之間的距離，使用該距離值去計算權重


In [8]:
# 讀入json檔
mrt_village_dis = pd.read_json('../data/mrt_village_dis.json', typ='series',encoding='utf-8')
mrt_village_dis.head(3)

頂埔站    {'新北市板橋區深丘里': 1.6, '新北市板橋區溪洲里': 2.995, '新北市板橋區...
永寧站    {'新北市板橋區深丘里': 0.527, '新北市板橋區溪洲里': 2.471, '新北市板...
土城站    {'新北市板橋區深丘里': 1.279, '新北市板橋區和平里': 2.859, '新北市板...
dtype: object

In [9]:
# 讀取彙整後的人口結構資料
pop = pd.read_csv('../data/pop_merge.csv',   encoding='utf-8')

# 將行政區里設為index 方便之後迴圈抓權重使用
pop['vill_name'] = pop['Country'] + pop['District'] + pop['Village']
pop.set_index('vill_name',inplace=True)
pop.head(3)

,Country,District,Village,Village_ID,Village_Chief,Address,lat,lng,納稅單位,綜合所得總額,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
vill_name,,,,,,,,,,,,,,,,,,,,,
基隆市中正區德義里,基隆市,中正區,德義里,10017010-001,張建源,基隆市中正區義二路97之 1號,25.132788,121.746435,275.0,23622.5,...,240,70,41,6,297,347,2.0,13.0,37,35
基隆市中正區正義里,基隆市,中正區,正義里,10017010-002,駱錫賢,基隆市中正區義二路147號,25.133882,121.747188,393.0,27012.9,...,396,166,128,13,516,588,3.0,19.0,65,57
基隆市中正區信義里,基隆市,中正區,信義里,10017010-003,江添丁,基隆市中正區正義路69號,25.134521,121.748392,257.0,19097.9,...,310,103,100,14,391,405,2.0,11.0,26,41


In [26]:
columns  = pop.columns[8:]   # 取出人口結構欄位名稱
dis_t    = np.exp(-dis)      # e^(-dis) 將距離取自然對數轉換
pop_list = []                # 裝pop_weighted
n_vill_list = []             # 裝鄰里辦公室數量


for i in range(len(mrt_village_dis)):
    
    stat_name = mrt_village_dis.index[i]   # 取所有車站名稱
    vill_list = mrt_village_dis[i]         # 取所有區里名稱       
    n_vill = len(vill_list)                # 取所有區里數量(距離車站n公里內)
    
    # 建立維度(1,35) 值為0的陣列準備承接權重計算結果
    pop_weighted = np.zeros((1,len(columns)))        # nparray 欄位數量取自人口結構

#     if i == 3: break    
#     print(f'{i+1:2d}, 里辦公室數:{n_vill:3d},{stat_name}')

    for j in range( n_vill ):
        vill_name = list(vill_list.keys())[j]    # 村里名  
        dis       = list(vill_list.values())[j]  # 捷運站與里辦公室距離
        
        
#         if j == 3: break        
#         print(f'\t{j+1},{vill_name} : {dis:.3f}')
        
        try:
            # 根據里辦公室與捷運站距離做權重轉換
            pop_weighted += ( pop.loc[vill_name,'納稅單位':]
                             .apply(lambda x: round(x*dis_t,0)) )  
             
        except Exception as e:
            print(f'{j}-{vill_name} : {ｅ}')
        
    try:
        pop_list.append(pop_weighted)
        n_vill_list.append(n_vill)
        
    except Exception as e:
        print(f'{i}-{stat_name}\t: {ｅ}')
    
    # check progess
    progress  = (i+1)/len(mrt_village_dis)*100
    print(f'progress ：{i+1}-{progress:.2f}％.',end='\r') # print out 完成 %數

# 將各車站轉換後的人口結構資料存為npArray格式
stat_pop = np.concatenate(pop_list, axis=0)

# 將各車站轉換後的人口結構資料存為Dataframe格式
df_stat_pop = pd.DataFrame(data = stat_pop ,    
#                       index = mrt_village_dis.index,
                      columns = columns)

df_stat_pop


df = pd.concat(
    [pd.DataFrame(mrt_village_dis.index, columns =['Station_Name'] ),  # 加入車站屬性與車站名
     pd.Series(n_vill_list, name='Num_Village'),                             # 加入捷運站週邊村里辦公室數量
     df_stat_pop ],                                                      # 加入主資料集 捷運站週邊權重後的人口結構  
    axis=1) 

df.to_csv('../data/station_pop_weighted.csv',index=False,encoding='utf-8')

In [27]:
df = pd.read_csv('../data/station_pop_weighted.csv', encoding='utf-8')
print(df.columns.values)

df

['Station_Name' 'Num_Village' '納稅單位' '綜合所得總額' '所得平均' '戶數' '人口' '男性人口'
 '女性人口' '0-14歲人口' '15-64歲人口' '65歲以上人口' '15-19歲人口' '20-24歲人口' '25-29歲人口'
 '30-34歲人口' '35-39歲人口' '40-44歲人口' '45-49歲人口' '50-54歲人口' '55-59歲人口'
 '60-64歲人口' '博士人口' '碩士人口' '大學院校人口' '專科人口' '高中職人口' '國中初職人口' '小學人口' '不識字人口'
 '未婚人口' '有偶人口' '出生數' '死亡數' '遷入人口' '遷出人口']


,Station_Name,Num_Village,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,女性人口,0-14歲人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
0,頂埔站,30,3628.0,305916.0,209.0,4763.0,12895.0,6417.0,6477.0,1678.0,...,3606.0,1605.0,1025.0,64.0,5762.0,5475.0,96.0,79.0,489.0,511.0
1,永寧站,49,6048.0,517197.0,348.0,7895.0,21287.0,10479.0,10808.0,2704.0,...,5779.0,2540.0,1614.0,100.0,9512.0,9097.0,163.0,116.0,818.0,823.0
2,土城站,63,8072.0,704197.0,452.0,10473.0,28175.0,13814.0,14361.0,3599.0,...,7543.0,3320.0,2109.0,128.0,12551.0,12079.0,215.0,147.0,1118.0,1077.0
3,海山站,105,12256.0,1082172.0,767.0,16110.0,42791.0,20954.0,21831.0,5336.0,...,11222.0,4929.0,3392.0,195.0,18845.0,18478.0,325.0,239.0,1694.0,1618.0
4,亞東醫院站,141,15563.0,1392627.0,1046.0,20761.0,54729.0,26732.0,27989.0,6987.0,...,13994.0,6109.0,4364.0,264.0,24080.0,23702.0,409.0,315.0,2123.0,2059.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,中壢站,66,7619.0,735040.0,530.0,10157.0,27016.0,13251.0,13764.0,4053.0,...,7058.0,2050.0,1625.0,108.0,11873.0,11736.0,267.0,154.0,1191.0,1112.0
288,埔心站,22,2719.0,244805.0,165.0,3456.0,9608.0,4772.0,4836.0,1565.0,...,2675.0,793.0,565.0,35.0,4221.0,4231.0,109.0,41.0,504.0,340.0
289,楊梅站,23,2235.0,198207.0,170.0,2783.0,7830.0,3906.0,3919.0,1209.0,...,2256.0,665.0,476.0,41.0,3426.0,3426.0,82.0,44.0,376.0,257.0
290,富岡站,4,317.0,25492.0,28.0,387.0,1154.0,602.0,551.0,139.0,...,346.0,114.0,113.0,18.0,473.0,535.0,13.0,8.0,39.0,31.0


### 車站資料探索

In [16]:
df[df['車站名稱'].str.contains(r'台北車?站')]

,車站種類,車站編號,車站名稱,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
11,tp_mrt,BL12,台北捷運台北車站,169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,tp_mrt,R10,台北捷運台北車站,169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203,ty_mrt,A1,桃園捷運台北車站,173,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
281,train,1000,台鐵台北站,168,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
322,thsr,Taipei,高鐵台北站,168,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df[df['車站名稱'].str.contains(r'板橋車?站')]

,車站種類,車站編號,車站名稱,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
6,tp_mrt,BL07,台北捷運板橋站,168,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,tp_mrt,Y16,台北捷運板橋站,167,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,train,1020,台鐵板橋站,165,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
323,thsr,Banqiao,高鐵板橋站,168,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df.describe()

,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,女性人口,0-14歲人口,15-64歲人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
count,336.000000,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,...,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0
mean,76.375000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,55.462365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,28.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,62.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,127.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,201.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 重製人口結構欄位

In [36]:
# 重製人口結構欄位
df = pd.read_csv('../data/station_pop_weighted.csv',encoding='utf-8')
# print(df.columns.values)

df['Income_Gross'] = df['綜合所得總額'].apply(lambda x :round(x/100,3))
df['Income_Average'] = df['綜合所得總額']/df['納稅單位']*10 

df['Population'] = df['人口']

df['Child']   = df['0-14歲人口']   / df['人口'] *100
df['Adult']   = df['15-64歲人口']  / df['人口'] *100
df['Old_Age']  = df['65歲以上人口'] / df['人口'] *100
df['Young_Adult']  = (df['20-24歲人口'] + df['25-29歲人口'] + df['30-34歲人口'] + df['35-39歲人口'] ) / df['人口'] *100
df['Middle_Age']  = (df['40-44歲人口'] + df['45-49歲人口'] + df['50-54歲人口'] + df['55-59歲人口'] + df['60-64歲人口']) / df['人口'] *100
df['Child_Dependency'] = df['0-14歲人口']   / df['15-64歲人口'] *100
df['Old_Dependency']   = df['65歲以上人口'] / df['15-64歲人口']  *100

df['Higher_Edu'] =  (df['博士人口'] + df['碩士人口'] + df['大學院校人口'])/ df['人口'] *100
df['Sec_Edu']    =  (df['高中職人口'] + df['專科人口'] + df['國中初職人口'])/ df['人口'] *100
df['Ele_Edu']    =  (df['小學人口'])/ df['人口'] *100
df['Uneducated'] =   df['不識字人口'] / df['人口'] *100

df['Male'] = df['男性人口'] / df['人口'] *100
df['Married'] = df['有偶人口'] / df['人口'] *100

df['Immigrant'] = df['遷入人口'] / df['人口']*100
df['Birth'] = df['出生數'] / df['人口']*100


# print(df.columns)

df =  pd.concat([df.loc[:,:'Num_Village'] ,
                  df.loc[:,'Income_Gross':].apply(lambda x :round(x,2))],
                  axis=1)
# print(df.columns)

df.to_csv('../data/station_pop_weighted_v3.csv',index=False,encoding='utf-8')

df

,Station_Name,Num_Village,Income_Gross,Income_Average,Population,Child,Adult,Old_Age,Young_Adult,Middle_Age,Child_Dependency,Old_Dependency,Higher_Edu,Sec_Edu,Ele_Edu,Uneducated,Male,Married,Immigrant,Birth
0,頂埔站,30,3059.16,843.21,12895.0,13.01,75.40,11.55,30.34,39.46,17.26,15.32,27.93,50.45,7.95,0.50,49.76,42.46,3.79,0.74
1,永寧站,49,5171.97,855.15,21287.0,12.70,75.72,11.56,30.71,39.60,16.78,15.26,29.49,49.60,7.58,0.47,49.23,42.74,3.84,0.77
2,土城站,63,7041.97,872.39,28175.0,12.77,75.67,11.55,30.69,39.66,16.88,15.26,29.80,49.34,7.49,0.45,49.03,42.87,3.97,0.76
3,海山站,105,10821.72,882.97,42791.0,12.47,74.44,13.08,29.96,39.32,16.75,17.57,30.29,48.71,7.93,0.46,48.97,43.18,3.96,0.76
4,亞東醫院站,141,13926.27,894.83,54729.0,12.77,73.31,13.92,29.31,38.94,17.41,18.99,30.77,47.87,7.97,0.48,48.84,43.31,3.88,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,中壢站,66,7350.40,964.75,27016.0,15.00,72.33,12.67,29.99,36.79,20.74,17.52,34.26,44.18,6.01,0.40,49.05,43.44,4.41,0.99
288,埔心站,22,2448.05,900.35,9608.0,16.29,73.84,9.86,32.26,35.72,22.06,13.35,31.20,46.25,5.88,0.36,49.67,44.04,5.25,1.13
289,楊梅站,23,1982.07,886.83,7830.0,15.44,73.44,11.05,31.14,36.27,21.03,15.04,30.57,47.31,6.08,0.52,49.89,43.75,4.80,1.05
290,富岡站,4,254.92,804.16,1154.0,12.05,72.27,15.51,30.42,35.96,16.67,21.46,27.73,48.61,9.79,1.56,52.17,46.36,3.38,1.13


In [35]:
df.describe()

,Num_Village,Income_Gross,Income_Average,Population,Child,Adult,Old_Age,Young_Adult,Middle_Age,Child_Dependency,Old_Dependency,Higher_Edu,Sec_Edu,Ele_Edu,Uneducated,Male,Married,Immigrant,Birth
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,72.554795,9882.954384,1010.097363,29395.160959,13.080548,71.659623,15.272842,27.879829,38.719247,18.256370,21.528253,34.191473,44.420068,7.453219,0.738322,48.865000,44.100548,4.701438,0.795788
std,54.554620,8456.866835,188.762909,21015.918448,2.076467,2.780239,3.859233,2.774929,1.731231,2.896616,6.324735,7.668101,5.764324,2.648040,1.017930,1.642428,1.555113,1.121900,0.192821
min,3.000000,16.340000,646.810000,133.000000,4.510000,63.910000,9.310000,15.790000,35.160000,7.060000,12.630000,11.650000,31.660000,3.850000,0.220000,46.610000,39.540000,1.480000,0.000000
25%,27.000000,2839.090000,860.922500,10780.250000,12.290000,70.067500,12.330000,26.180000,37.640000,16.805000,16.790000,29.025000,39.842500,5.927500,0.390000,47.827500,42.960000,3.970000,0.737500
50%,57.000000,7536.825000,954.830000,25476.500000,13.220000,72.335000,14.190000,28.270000,38.565000,18.400000,19.440000,33.930000,45.620000,7.105000,0.470000,48.735000,43.740000,4.535000,0.780000
75%,118.500000,14562.717500,1160.427500,49456.500000,14.335000,73.570000,17.975000,29.952500,39.570000,20.275000,25.765000,39.930000,48.672500,8.202500,0.560000,49.315000,45.385000,5.120000,0.890000
max,201.000000,34337.390000,1449.760000,74150.000000,17.480000,76.700000,31.580000,33.210000,45.860000,24.440000,49.410000,49.680000,60.190000,20.300000,6.310000,57.140000,49.080000,11.610000,1.340000
